<a href="https://colab.research.google.com/github/CandyEatingReporter/DHP/blob/main/Bluesky_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U datasets fsspec


from datasets import load_dataset
import re

# ✅ Correct way to load a Hugging Face-hosted dataset
  dataset = load_dataset("withalim/bluesky-posts", split="train")

In [ ]:
dataset[0]

{'text': 'yo',
 'created_at': '1001-01-11T01:11:11.804Z',
 'author': 'jarivi.bsky.social',
 'uri': 'at://did:plc:kblt2p6hbypqurirkc723vht/app.bsky.feed.post/3lc4iv3krp22j',
 'has_images': False,
 'reply_to': None}

In [ ]:
print(len(dataset))

7877660


In [ ]:
# Precompiled regex pattern (case-insensitive)
pattern = re.compile(r"(ai|artificial intelligence)", re.IGNORECASE)

# Collect matching posts
filtered_posts = []
max_results = 10000  # adjust as needed
post_count = 0
for post in dataset:
    text = post.get("text", "")
    if pattern.search(text):
        filtered_posts.append(post)
    if len(filtered_posts) >= max_results:
        break
    post_count += 1
    if post_count % 1000 == 0:
        print(f"Processed {post_count} posts.")

print(f"Collected {len(filtered_posts)} matching posts.")


Processed 1000 posts.
Processed 2000 posts.
Processed 3000 posts.
Processed 4000 posts.
Processed 5000 posts.
Processed 6000 posts.
Processed 7000 posts.
Processed 8000 posts.
Processed 9000 posts.
Processed 10000 posts.
Processed 11000 posts.
Processed 12000 posts.
Processed 13000 posts.
Processed 14000 posts.
Processed 15000 posts.
Processed 16000 posts.
Processed 17000 posts.
Processed 18000 posts.
Processed 19000 posts.
Processed 20000 posts.
Processed 21000 posts.
Processed 22000 posts.
Processed 23000 posts.
Processed 24000 posts.
Processed 25000 posts.
Processed 26000 posts.
Processed 27000 posts.
Processed 28000 posts.
Processed 29000 posts.
Processed 30000 posts.
Processed 31000 posts.
Processed 32000 posts.
Processed 33000 posts.
Processed 34000 posts.
Processed 35000 posts.
Processed 36000 posts.
Processed 37000 posts.
Processed 38000 posts.
Processed 39000 posts.
Processed 40000 posts.
Processed 41000 posts.
Processed 42000 posts.
Processed 43000 posts.
Processed 44000 post

In [ ]:
# OPTIONAL: prompt: Mount a google drive directory to store data

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import json

with open("dh_hashtag_posts.jsonl", "w") as f:
    for post in filtered_posts:
        f.write(json.dumps(post) + "\n")

In [ ]:
import json
import re
from collections import defaultdict

# Load your filtered posts
with open("dh_hashtag_posts.jsonl") as f:
    posts = [json.loads(line) for line in f]

nodes = {}
edges = defaultdict(int)

for post in posts:
    # Check if 'author' key exists and use its value directly as the handle
    if "author" in post:
        author = post["author"]
        # Ensure author is a string before proceeding
        if isinstance(author, str):
            text = post.get("text", "")
            mentions = re.findall(r"@([a-zA-Z0-9_.-]+)", text)

            nodes[author] = True
            for mention in mentions:
                nodes[mention] = True
                edge_key = tuple(sorted([author, mention]))  # undirected
                edges[edge_key] += 1
        # else:
            # Optionally handle cases where 'author' exists but is not a string
            # print(f"Skipping post with non-string author: {post}")
    # else:
        # Optionally handle posts without an 'author' key
        # print(f"Skipping post with no author: {post}")


# Build graph
graph = {
    "nodes": [{"id": u} for u in nodes],
    "links": [{"source": a, "target": b, "value": w} for (a, b), w in edges.items()]
}

with open("graph.json", "w") as f:
    json.dump(graph, f, indent=2)

In [ ]:
import json
from IPython.display import HTML

# Keep it small for demo (or filter)
small_graph = {
    "nodes": graph["nodes"][:50],
    "links": [l for l in graph["links"] if l["source"] in {n["id"] for n in graph["nodes"][:50]} and l["target"] in {n["id"] for n in graph["nodes"][:50]}]
}

graph_data = json.dumps(small_graph)


In [ ]:
# prompt: # Get a sample of the data from the top of graph.json

# Load the JSON data
with open('graph.json', 'r') as f:
    data = json.load(f)

# Get the first few items
sample_nodes = data['nodes'][:10]  # Sample first 10 nodes
sample_links = data['links'][:10]  # Sample first 10 links

# Print the sample
print("Sample Nodes:")
print(json.dumps(sample_nodes, indent=2))

print("\nSample Links:")
print(json.dumps(sample_links, indent=2))

Sample Nodes:
[
  {
    "id": "leiarcaica5.bsky.social"
  },
  {
    "id": "leiarcaica4.bsky.social"
  },
  {
    "id": "jestamusic.bsky.social"
  },
  {
    "id": "meggan.bsky.social"
  },
  {
    "id": "d365geektest.bsky.social"
  },
  {
    "id": "nateritter.com"
  },
  {
    "id": "joec0914"
  },
  {
    "id": "ashmaurya.bsky.social"
  },
  {
    "id": "mrbula.bsky.social"
  },
  {
    "id": "missrogue"
  }
]

Sample Links:
[
  {
    "source": "leiarcaica4.bsky.social",
    "target": "leiarcaica5.bsky.social",
    "value": 2
  },
  {
    "source": "joec0914",
    "target": "nateritter.com",
    "value": 1
  },
  {
    "source": "missrogue",
    "target": "nateritter.com",
    "value": 1
  },
  {
    "source": "mediaphyter.",
    "target": "nateritter.com",
    "value": 1
  },
  {
    "source": "humanlever",
    "target": "nateritter.com",
    "value": 1
  },
  {
    "source": "OnkelSchark",
    "target": "nateritter.com",
    "value": 1
  },
  {
    "source": "factoryjoe",
    "tar

In [ ]:
import json
from IPython.display import HTML

# Load graph from file (use your full path if needed)
with open("graph.json") as f:
    graph_data = json.load(f)

# Optional: filter for a smaller graph to avoid crashing the browser
subset_nodes = graph_data["nodes"][:100]
subset_ids = {n["id"] for n in subset_nodes}
subset_links = [l for l in graph_data["links"] if l["source"] in subset_ids and l["target"] in subset_ids]

subset_graph = {
    "nodes": subset_nodes,
    "links": subset_links
}

# Convert to JSON string for JS
graph_json = json.dumps(subset_graph)


In [ ]:
from IPython.display import HTML
import json

with open("graph.json") as f:
    full_graph = json.load(f)

# Optional: subset for performance
subset_nodes = full_graph["nodes"][:200]
subset_ids = {n["id"] for n in subset_nodes}
subset_links = [l for l in full_graph["links"] if l["source"] in subset_ids and l["target"] in subset_ids]
subset_graph = {"nodes": subset_nodes, "links": subset_links}

graph_json = json.dumps(subset_graph)

HTML(f"""
<div id="graph" style="width: 100%; height: 700px; border:1px solid #ccc;"></div>
<script src="https://d3js.org/d3.v7.min.js"></script>
<script>
  const graph = {graph_json};
  const width = window.innerWidth - 50;
  const height = 700;

  const svg = d3.select("#graph")
    .append("svg")
    .attr("width", width)
    .attr("height", height)
    .call(d3.zoom().on("zoom", (event) => {{
      container.attr("transform", event.transform);
    }}))
    .append("g");

  const container = svg.append("g");

  const link = container.append("g")
    .selectAll("line")
    .data(graph.links)
    .join("line")
    .attr("stroke", "#aaa")
    .attr("stroke-width", d => Math.sqrt(d.value || 1));

  const node = container.append("g")
    .selectAll("circle")
    .data(graph.nodes)
    .join("circle")
    .attr("r", 5)
    .attr("fill", "steelblue")
    .call(d3.drag()
      .on("start", dragstart)
      .on("drag", dragged)
      .on("end", dragend));

  node.append("title").text(d => d.id);

  const simulation = d3.forceSimulation(graph.nodes)
    .force("link", d3.forceLink(graph.links).id(d => d.id).distance(40))
    .force("charge", d3.forceManyBody().strength(-50))
    .force("center", d3.forceCenter(width / 2, height / 2))
    .force("collide", d3.forceCollide(10));

  simulation.on("tick", () => {{
    link
      .attr("x1", d => d.source.x)
      .attr("y1", d => d.source.y)
      .attr("x2", d => d.target.x)
      .attr("y2", d => d.target.y);

    node
      .attr("cx", d => d.x)
      .attr("cy", d => d.y);
  }});

  function dragstart(event) {{
    if (!event.active) simulation.alphaTarget(0.3).restart();
    event.subject.fx = event.subject.x;
    event.subject.fy = event.subject.y;
  }}

  function dragged(event) {{
    event.subject.fx = event.x;
    event.subject.fy = event.y;
  }}

  function dragend(event) {{
    if (!event.active) simulation.alphaTarget(0);
    event.subject.fx = null;
    event.subject.fy = null;
  }}
</script>
""")


This cell processes the graph data from the "graph.json" file to prepare it for visualization using the Cytoscape.js library.

1. **Load and Clean Data**: It reads the graph data from "graph.json", then cleans it by ensuring node IDs are valid strings and that links connect to valid nodes.
1. **Subset Data**: To improve performance, it creates a smaller subset of the data by taking the first 100 valid nodes and all valid links connected to these nodes.
1. **Format for Cytoscape.js**: The subsetted data is then converted into a list of elements (nodes and edges) with a structure that Cytoscape.js expects.
1. **Create JSON String**: Finally, this list of elements is serialized into a JSON string, which will be used in the next cell to render the interactive graph visualization.

In [ ]:
from IPython.display import HTML
import json
import re

# --- Load and clean ---
with open("graph.json") as f:
    raw_graph = json.load(f)

valid_nodes = [n for n in raw_graph["nodes"] if isinstance(n["id"], str) and n["id"].strip()]
id_set = {n["id"] for n in valid_nodes}
valid_links = [l for l in raw_graph["links"] if l["source"] in id_set and l["target"] in id_set]

# --- Subset ---
subset_nodes = valid_nodes[:100]
subset_ids = {n["id"] for n in subset_nodes}
subset_links = [l for l in valid_links if l["source"] in subset_ids and l["target"] in subset_ids]

# --- Build elements ---
cy_elements = []
for n in subset_nodes:
    cy_elements.append({ "data": { "id": n["id"] } })

for l in subset_links:
    cy_elements.append({
        "data": {
            "id": f"{l['source']}__{l['target']}",
            "source": l["source"],
            "target": l["target"]
        }
    })

cy_json = json.dumps(cy_elements)


## Graph Visualization using Cytoscape.js
This cell renders an interactive graph visualization directly in the notebook output using the Cytoscape.js JavaScript library.

It takes the processed graph data (nodes and edges) generated in the previous steps and embeds it within an HTML container. Cytoscape.js is then initialized with this data, applying predefined styles for nodes and edges and using a force-directed layout ('cose') to arrange the graph elements.

In [ ]:
HTML(f"""
<div id="cy" style="width: 100%; height: 600px; border: 1px solid #aaa;"></div>
<script src="https://unpkg.com/cytoscape@3.24.0/dist/cytoscape.min.js"></script>
<script>
try {{
  const elements = {cy_json};
  const cy = cytoscape({{
    container: document.getElementById('cy'),
    elements: elements,
    style: [
      {{
        selector: 'node',
        style: {{
          'label': 'data(id)',
          'width': 10,
          'height': 10,
          'font-size': 5,
          'background-color': '#0074D9',
          'text-valign': 'center',
          'text-halign': 'center',
          'color': '#fff'
        }}
      }},
      {{
        selector: 'edge',
        style: {{
          'width': 1,
          'line-color': '#ccc',
          'opacity': 0.7
        }}
      }}
    ],
    layout: {{
      name: 'cose',
      animate: true,
      fit: true,
      padding: 10
    }}
  }});

  console.log("Cytoscape loaded successfully with", elements.length, "elements.");
}} catch (err) {{
  console.error("Cytoscape error:", err);
}}
</script>
""")


## Python Preprocessing (Hashtag Co-occurrence)
This builds a bipartite graph of users and hashtags, then collapses it into co-occurrence edges between hashtags.

In [ ]:
import json
import re
from collections import defaultdict
from IPython.display import HTML

with open("dh_hashtag_posts.jsonl") as f:
    posts = [json.loads(line) for line in f]

tag_pairs = defaultdict(int)
tag_freq = defaultdict(int)
limit = 10000

for post in posts[:limit]:
    text = post.get("text", "")
    tags = sorted(set(re.findall(r"#(\w+)", text.lower())))  # ✅ FIXED regex
    full_tags = [f"#{tag}" for tag in tags]

    for tag in full_tags:
        tag_freq[tag] += 1

    for i in range(len(full_tags)):
        for j in range(i + 1, len(full_tags)):
            pair = tuple(sorted([full_tags[i], full_tags[j]]))
            tag_pairs[pair] += 1

# Filter edges for signal clarity
min_weight = 2
cy_elements = []
used_tags = set()

for (t1, t2), weight in tag_pairs.items():
    if weight >= min_weight:
        cy_elements.append({
            "data": {
                "id": f"{t1}__{t2}",
                "source": t1,
                "target": t2,
                "weight": weight
            }
        })
        used_tags.update([t1, t2])

# Add only connected nodes
for tag in used_tags:
    cy_elements.append({
        "data": {
            "id": tag,
            "freq": tag_freq[tag]
        }
    })

print("Final Nodes:", len(used_tags), "Edges:", len(cy_elements) - len(used_tags))
cy_json = json.dumps(cy_elements)


Final Nodes: 65 Edges: 47


## Cytoscape.js Visualization Cell (with `fcose` layout)
This cell includes dynamic script loading for the `fcose` layout:

In [ ]:
HTML(f"""
<div id="cy" style="width: 100%; height: 700px; border:1px solid #aaa;"></div>

<script src="https://unpkg.com/cytoscape@3.22.0/dist/cytoscape.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/cytoscape-fcose@2.2.1/cytoscape-fcose.min.js"></script>

<script>
(function() {{
  try {{
    if (typeof cytoscape === "undefined") {{
      console.error("Cytoscape failed to load");
      return;
    }}

    cytoscape.use(window.fcose);

    const elements = {cy_json};
    console.log("Loaded elements:", elements.length);

    const cy = cytoscape({{
      container: document.getElementById('cy'),
      elements: elements,
      style: [
        {{
          selector: 'node',
          style: {{
            'label': 'data(id)',
            'width': 'mapData(freq, 1, 50, 8, 20)',
            'height': 'mapData(freq, 1, 50, 8, 20)',
            'font-size': 6,
            'background-color': '#0074D9',
            'color': '#fff',
            'text-valign': 'center',
            'text-halign': 'center'
          }}
        }},
        {{
          selector: 'edge',
          style: {{
            'width': 'mapData(weight, 1, 10, 1, 5)',
            'line-color': '#bbb',
            'opacity': 0.7
          }}
        }}
      ],
      layout: {{
        name: 'fcose',
        fit: true,
        padding: 50,
        animate: true,
        randomize: true,
        nodeRepulsion: 30000,
        idealEdgeLength: 100,
        edgeElasticity: 0.1,
        gravity: 0.2,
        numIter: 1000
      }}
    }});
  }} catch (e) {{
    console.error("Cytoscape error:", e);
  }}
}})();
</script>
""")
